In [1]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import SystemMessage

In [ ]:
OPENAI_API_KEY = "##"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
loader = PyPDFLoader("manual_avancado_nobreak_fxp2000.pdf")
documents = loader.load()

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents,embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
chat = ChatOpenAI(model='gpt-4o', temperature=0)

In [ ]:
system_message = SystemMessage(content=('''
Você é um assistente virtual especializado no atendimento ao cliente no nobreak FXP-2000.
Responsa claramente perguntas técnicas, funcionalidades, garantia, manutenção, atualizações e suporte técnico.
Se a pergunta for irrelevante, responda educadamente recusando a pergunta.
'''))

In [ ]:
store = {}
def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

qa_chain = ConversationalRetrievalChain.from_llm(
    llm = chat,
    retriever = retriever,
    return_source_documents = False
)

chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

In [ ]:
session_id = "sessao1"
while True:
  pergunta = input("Você: ")
  if pergunta.lower() in ["sair","quit","exit"]:
    print("Até logo!")
    break
  resposta = chain_with_history.invoke(
      {"question": pergunta},
      config={"configurable":{"session_id": session_id}}
  )
  print("Assistente: ", resposta["answer"])
